In [116]:
from keras.models import load_model
import numpy as np
import pandas as pd
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tqdm import tqdm_notebook
import time
labels_to_names = { 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'} 

In [172]:
model = models.load_model('keras-retinanet/50model.h5', backbone_name='resnet50')

/home/elsa/anaconda3/envs/Tensorflow/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [173]:
filename = []
lab = []
xmin = []
ymin = []
xmax = []
ymax = []
for i in tqdm_notebook(os.listdir('test/')):
    image = read_image_bgr('test/'+i)


    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        #print(b,label)
        filename.append(i)
        lab.append(label)
        xmin.append(b[0])
        ymin.append(b[1])
        xmax.append(b[2])
        ymax.append(b[3])
   

In [133]:
np.concatenate([np.array(filename),np.array(lab),np.array(xmin),
                             np.array(ymin),np.array(xmax),np.array(ymax)],axis = 1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [179]:
counter = 0
with open('result.txt', mode='a') as f:
            f.write('<annotations>\n')
while counter<2375:
    i = filename[counter]
    obj = sum(np.array(filename)==i)
    title = '<annotation>\n    <filename>{}</filename>'.format(i)
    with open('result.txt', mode='a') as f:
        f.write(title)
    for k in range(obj):
        box='\n    <object>\n        <name>{}</name>\n        <bndbox>\n            <xmin>{}</xmin>\n            <ymin>{}</ymin>\n            <xmax>{}</xmax>\n            <ymax>{}</ymax>\n        </bndbox>\n    </object>\n'.format(lab[counter+k],xmin[counter+k],ymin[counter+k],xmax[counter+k],ymax[counter+k])
        with open('result.txt', mode='a') as f:
            f.write(box)

    with open('result.txt', mode='a') as f:
            f.write('</annotation>\n')
    counter += obj
with open('result.txt', mode='a') as f:
            f.write('</annotations>\n')

In [178]:
len(filename)

2375